In [ ]:
from togglwrapper import Toggl
import pandas as pd
import datetime as dt
from IPython.core.display import display, HTML, display_javascript, Javascript
TOGGL_API_TOKEN='188b4b533d58101a5fcc800f9f00569d'
TOGGL_WORKSPACE_ID=57456
NUMBER_OF_TOGGL_DATA_TO_PULL=30
toggl = Toggl(TOGGL_API_TOKEN)

In [ ]:
htm="""
<!DOCTYPE html>
<html lang="en-US">
<script type="text/javascript" src="https://www.gstatic.com/charts/loader.js"></script>

<script type="text/javascript">
// Load google charts
google.charts.load('current', {'packages':['corechart']});
google.charts.setOnLoadCallback(drawChart);

// Draw the chart and set the chart values
function drawChart() {
  var data = google.visualization.arrayToDataTable([
  ['Task', 'Hours per Day'],
  ['Work', 8],
  ['Eat', 2],
  ['TV', 4],
  ['Gym', 2],
  ['Sleep', 8]
]);

  // Optional; add a title and set the width and height of the chart
  var options = {'title':'My Average Day', 'width':550, 'height':400};

  // Display the chart inside the <div> element with id="piechart"
  var chart = new google.visualization.PieChart(document.getElementById('piechart'));
  chart.draw(data, options);
   var chart2 = new google.visualization.PieChart(document.getElementById('piechart2'));
  chart2.draw(data, options);
}
</script>

<body>

<h1>My Web Page</h1>
<table width='90%'><tr><td><div id="piechart"></div></td><td><div id="piechart2"></div></td></tr></table>

</body>
</html>
"""
display(HTML(htm))

In [ ]:
# Pull Toggl Data
reference_date=dt.datetime.strptime("2017-05-01","%Y-%m-%d")
#reference_date=dt.datetime.now()
start_date=str(reference_date.date()-dt.timedelta(days=NUMBER_OF_TOGGL_DATA_TO_PULL))+"T00:00:00+00:00"
end_date=str(reference_date.date())+"T00:00:00+00:00"

# Pull projects
projects=pd.DataFrame(data=toggl.Workspaces.get_projects(TOGGL_WORKSPACE_ID))
projects=projects.loc[:,['id','name']]

# Pull time entries
time_entries=pd.DataFrame(data=toggl.TimeEntries.get(start_date=start_date,end_date=end_date))
time_entries.stop=pd.to_datetime(time_entries.stop)
time_entries.start=pd.to_datetime(time_entries.start)
time_entries=time_entries.loc[:,['start','description','duration','pid']]
tasks=pd.merge(time_entries,projects,how='left',left_on='pid',right_on='id')
tasks=tasks.loc[:,['start','name','description','duration']]
tasks

In [ ]:
import pandas as pd
import datetime as dt
#import my_util
import matplotlib.pyplot as plt
import ast
from IPython.core.display import display, HTML, display_javascript, Javascript
from togglwrapper import Toggl
import simplenote as sn
from io import StringIO

In [14]:
simplenote = sn.Simplenote('mohdjamal8453@gmail.com', 'simple123')

In [20]:
task_tracker_config_note_key='58cb45a3-54ac-4316-9a57-7272b8edc0a3'
task_tracker_config_note_raw_content=simplenote.get_note(task_tracker_config_note_key)[0]['content']

In [33]:
task_tracker_config_note_raw_content.splitlines()[3]

"* daily_goal={'meeting':1.5,'social':0.5,'comp_namaaz':0.5,'goal':2,'careem':3} # overall effiency is tracked, these are items of current focus and are logged, later on they may not be tracked or logged"

In [36]:
daily_goal=1

In [37]:
x=task_tracker_config_note_raw_content.splitlines()[3].replace("*","").strip()
exec(x[:x.index("#")])
daily_goal

{'careem': 3, 'comp_namaaz': 0.5, 'goal': 2, 'meeting': 1.5, 'social': 0.5}

In [43]:
x= "hhsjhasjha sjlasj"
x.("#",)

ValueError: substring not found

In [80]:
def get_task_tracker_config():
    task_tracker_config_note_key = '58cb45a3-54ac-4316-9a57-7272b8edc0a3'
    task_tracker_config_note_raw_content = simplenote.get_note(task_tracker_config_note_key)[0]['content']
    task_tracker_config={}
    for code_str in task_tracker_config_note_raw_content.splitlines():
        code_str=code_str.replace("*","").strip()
        if(code_str.count("=")==0):
            continue
        if(code_str.count("#")==0):
            code_str=code_str[:code_str.index("#")] # remove comments
        equalto_index=code_str.index("=")
        var_name=code_str[:equalto_index]
        var_value=eval(code_str[equalto_index+1:])
        task_tracker_config[var_name]=var_value
    return task_tracker_config
    
get_task_tracker_config()

{'daily_goal': {'careem': 3,
  'comp_namaaz': 0.5,
  'goal': 2,
  'meeting': 1.5,
  'social': 0.5},
 'ft_projects': ['quran', 'ilm', 'careem', 'du'],
 'weekly_goal': {'goal': 14, 'ilm': 7, 'quran': 7}}

In [24]:
eval(task_tracker_config_note_raw_content.splitlines()[3].replace("*","").strip())

SyntaxError: invalid syntax (<string>, line 1)

In [ ]:
import datetime as dt
reference_date=dt.datetime.now()
#reference_date=dt.datetime.strptime("2017/11/29","%Y/%m/%d")
calculate_n_display_task_n_goal_metrics(reference_date)